In [2]:
# dimension
n = 64
# plaintext modulus
p = 257
# ciphertext modulus
q = 1048583

V = VectorSpace(GF(q), n)
S = V.random_element()

def encrypt(m):
    A = V.random_element()
    e = randint(-1, 1)
    b = A * S + m + p * e
    return A, b

In [7]:
As = []
bs = []
for i in range(30):
    A = V.random_element()
    b = S * A + randint(-1, 1)
    As.append(A)
    bs.append(b)

$b_0 = \vec{A_0} * \vec{S} + e_0 \pmod{q}$

$b_1 = \vec{A_1} * \vec{S} + e_1 \pmod{q}$

...

$b_m = \vec{A_m} * \vec{S} + e_m \pmod{q}$


$$M = \begin{pmatrix}
\vec{A_0}  \\
\vec{A_1} \\
... \\
\vec{A_m} \\
\end{pmatrix}$$

$\vec{b} = M * \vec{S} + \vec{e} \pmod{q}$

$\vec{b} = M * \vec{S} + \vec{e} + (K_1, K_2, K_3, ..., K_m) * q$


$\vec{A_0} = (a_{00}, a_{01}, ..., a_{0n})$

$$M = \begin{pmatrix}
a_{00} & a_{10} & ... & a_{m0}  \\
a_{01} & a_{11} & ... & a_{m1}  \\
... \\
a_{0n} & a_{1n} & ... & a_{mn}  \\
q & 0 & 0 ... & 0 \\
0 & q & 0 ... & 0 \\
... \\
0 & 0 & 0 ... & q \\
\end{pmatrix}$$


$\vec{k} = (s_0, s_1, s_2, ..., s_n, K_1, K_2, ..., K_m)$

$\vec{k} * M = (e_0, e_1, e_2, ..., e_m) \approx 2^m$

In [29]:
def kg(blocksize: int, nbit=3):
    key = [getRandomNBitInteger(nbit)]
    for i in range(blocksize - 1):
        z = key[-1]
        key.append(randint(2 * z, 2 * z + getrandbits(nbit)))
    return key


# K = (k_0, k_1, ..., k_n)
# Pk = (k_0 * A, k_1 * A, ..., k_n * A) % B

def keygen(blocksize, nbit=3):
    r = kg(blocksize, nbit)
    B = randint(2 * r[-1] + 1, 2 * r[-1] + getrandbits(nbit))
    A = randint(0, B)
    while gcd(A, B) != 1:
        A = randint(0, B)
    pk = list()
    for i in r:
        pk.append((i * A) % B)
    return ((A, B, r), (pk, B))

# (m_0, m_1, .., m_n) * vec(Pk)


def encrypt(m: list, pubkey):
    K_seq, Mod = pubkey
    s = 0
    for i, j in zip(m, K_seq):
        s += i * j
        s %= Mod
    return s

def deq(s: int, key: list):
    m = list()
    for i in range(len(key) - 1, -1, -1):
        if s >= key[i]:
            m = [1] + m
            s -= key[i]
        else:
            m = [0] + m
    if s != 0:
        raise "bad word"
    return m

def decrypt(s, prikey):
    A, B, r = prikey
    s = (pow(A, -1, B) * s) % B
    m = deq(s, r)
    return m

In [30]:
sk, pk = keygen(5, 10)
pk

([29994, 34891, 17159, 13859, 5585], 37954)

In [37]:
m = [0, 1, 1, 0, 1]
ct = encrypt(m, pk)

In [38]:
ct

19681

In [39]:
decrypt(ct, sk)

[0, 1, 1, 0, 1]

In [55]:
M = Matrix(7)

M.set_block(0, 0, identity_matrix(6))
M.set_block(0, 0, identity_matrix(5) * 2)
M.set_block(0, 6, Matrix([pk[0] + [pk[1]] + [-int(ct)]]).T)
M.set_block(6, 0, Matrix([-1] * 6))
M

[     2      0      0      0      0      0  29994]
[     0      2      0      0      0      0  34891]
[     0      0      2      0      0      0  17159]
[     0      0      0      2      0      0  13859]
[     0      0      0      0      2      0   5585]
[     0      0      0      0      0      1  37954]
[    -1     -1     -1     -1     -1     -1 -19681]

In [57]:
M.solve_left(M.LLL()[0])

(0, 1, 1, 0, 1, -1, 1)

In [63]:
def keygen(nbits: int):
    q = random_prime(2**nbits)
    while True:
        f = randint(1, floor(sqrt(q / 2)))
        g = randint(ceil(sqrt(q / 4)), floor(sqrt(q / 2)))
        if gcd(f, q * g) == 1:
            break

    h = (pow(f, -1, q) * g) % q
    return (f, g), (h, q)


def encrypt(h, q, m):
    assert m < sqrt(q / 4)

    r = randint(0, floor(sqrt(q / 2)))
    return (r * h + m) % q


def decrypt(f, g, q, e):
    a = (f * e) % q
    b = (pow(f, -1, g) * a) % g
    return b

In [64]:
sk, pk = keygen(512)
m = randint(0, isqrt(pk[1] // 4))

In [65]:
ct = encrypt(*pk, m)

In [66]:
ct

6405455233443648413302990372677665839649028430858868236751114679896053184379250081145537036316547467634621807840656798125073898562489851590582794671492704

In [67]:
decrypt(*sk, pk[1], ct)

28156443611414235651282046453914603220410073985342673458136042479643044947532

In [68]:
m

28156443611414235651282046453914603220410073985342673458136042479643044947532

$h * f \equiv g \pmod{q}$

$h * f - K * q  = g$

$f * (1, h) - K * (0, q) = (F, G)$

$f - K * 0 = f$

$f * h - K * q = g$

$$M = \begin{pmatrix}
1 & h \\ 
0 & -q \\
\end{pmatrix}$$


$\vec{k} = (f, K)$

$\vec{k} * M = (f, f * h - K * q) = (f, g)$

$|(f, g)| < \sqrt(q/2 + q/2) = \sqrt{q}$

In [69]:
M = Matrix([[1, pk[0]], [0, pk[1]]])

L = M.LLL()

In [74]:
abs(int(L[0][0])).bit_length()

256

In [81]:
f, g = L[0]
f, g

(-74012543077019476278607737242217003101972402804824519314294985030168484510892,
 -70285935410572241048328880979525176584570718264454548677249287123609989634261)

In [79]:
sk

(74012543077019476278607737242217003101972402804824519314294985030168484510892,
 70285935410572241048328880979525176584570718264454548677249287123609989634261)

In [82]:
def HadamardRatio(v):
    m = Matrix(v)
    prod = 1.0
    for i in range(len(v)):
        prod *= v[i].norm()
    return pow(abs(m.det()) / prod, 1 / len(v))


def nearest(x):
    if abs(floor(x) - x) < 0.5:
        return floor(x)
    return ceil(x)


def Babai_algorithm(base, vec):
    m = Matrix(base)
    sol = m.solve_left(vec)
    sol = vector([nearest(x) for x in sol])
    ans = zero_vector(len(base))
    for i in range(len(base)):
        ans += vector(base[i]) * sol[i]
    return ans


def get_sum_rows_m(i, j, a=1, b=1, dim=0):
    m = identity_matrix(dim)
    m[i, i] = a
    m[i, j] = b
    return m


def get_permutation_rows_m(i, j, dim):
    m = identity_matrix(dim)
    m[i, i] = 0
    m[j, j] = 0
    m[i, j] = 1
    m[j, i] = 1
    return m


def get_secret(dim: int, m):
    I = identity_matrix(dim)
    for i in range(100):
        m1 = get_sum_rows_m(randint(0, dim - 1), randint(0, dim - 1), 1, 1, dim)
        m2 = get_permutation_rows_m(randint(0, dim - 1), randint(0, dim - 1), dim)
        m = m1 * m2 * m
    while True:
        if 0 < HadamardRatio([vector(m.row(x)) for x in range(dim)]) < 0.01:
            return [vector(m.row(x)) for x in range(dim)]
        m1 = get_sum_rows_m(randint(0, dim - 1), randint(0, dim - 1), 1, 1, dim)
        m2 = get_permutation_rows_m(randint(0, dim - 1), randint(0, dim - 1), dim)
        m = m1 * m2 * m


def keygen(dim: int, d: int):
    while True:
        vs = list()
        for i in range(dim):
            vi = list()
            for i in range(dim):
                vi.append(randint(-d, d))
            vi = vector(vi)
            vs.append(vi)
        if HadamardRatio(vs) >= 0.90:
            break

    priv = vs
    pub = get_secret(dim, Matrix(priv))
    return (pub, randint(0, 50)), priv


def encrypt(pub, binm, flag):
    base, sig = pub
    r = vector([randint(-sig, sig) for i in range(len(base))])
    print(f"Random: {r}")
    if flag:
        for i, j in zip(binm, base):
            r += i * j
        return r
    else:
        for i, j in zip(r, base):
            m += i * j
        return m


def decrypt(priv, pub, c):
    w = Matrix(priv)
    v = Babai_algorithm(priv, c)
    v1 = Babai_algorithm(pub[0], c)
    print(f"Babai solved: {v}")
    print(f"Babai bad {v1}")
    print(f"R = {c - v}")
    m = Matrix(pub[0]).solve_left(v)
    m1 = Matrix(pub[0]).solve_left(v1)
    return m, m1

In [84]:
P = PolynomialRing(GF(101), "x")
x = P.gens()[0]
while True:
    f = P.random_element(degree=5)
    if f.is_irreducible():
        break

In [87]:
Q = P.quotient(f.monic())

Univariate Quotient Polynomial Ring in xbar over Finite Field of size 101 with modulus x^5 + 35*x^4 + 88*x^3 + 82*x^2 + 50*x + 27